In [2]:
!git clone https://github.com/GroupLe/grouple-face-tagger/

Cloning into 'grouple-face-tagger'...
remote: Enumerating objects: 397, done.
remote: Counting objects: 100% (397/397), done.
remote: Compressing objects: 100% (291/291), done.
remote: Total 397 (delta 142), reused 326 (delta 79), pack-reused 0
Receiving objects: 100% (397/397), 13.88 MiB | 26.56 MiB/s, done.
Resolving deltas: 100% (142/142), done.


In [3]:
!cd grouple-face-tagger && touch __init__.py
!mkdir data

In [9]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import os
import matplotlib.pyplot as plt
from pathlib import Path
import torch
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import Dataset
import torchvision
import wandb
# import sys
# sys.path.insert(0, "grouple-face-tagger/models/face_detection/")
from snn_model.datasets import TripletPathDataset
from snn_model.model import EmbeddingNet, TripletNet
from snn_model.functions import accuracy
from snn_model.transformations import EmptyTransformation
from statistics import mean

In [2]:
root = Path('../../data/face_detection/processed_test')

In [3]:
filecount=0
for dirpath, dirs, files in os.walk(root):
    for filename in files:
        filecount+=1
print(filecount)
print(len(os.listdir(root)))

1659
23


In [4]:
import torchvision.transforms as T

composed = torchvision.transforms.Compose([T.ToTensor(),
                                           T.Resize((128, 128)),
                                           T.RandomChoice((T.ColorJitter(0.05, 0.05, 0.05),
                                                          T.RandomRotation(degrees=(0, 30)),
                                                          EmptyTransformation(),
                                                          EmptyTransformation(),
                                                          EmptyTransformation()))])
                                                          

In [5]:
siamse_dataset = TripletPathDataset(Path(root), transform = composed)

23it [00:04,  4.96it/s]


In [6]:
print(round(len(siamse_dataset)*0.8))
print(int(len(siamse_dataset)*0.2))

siamse_dataset_train, siamse_dataset_test = torch.utils.data.random_split(siamse_dataset,
                                                                         (round(len(siamse_dataset)*0.8),
                                                                          round(len(siamse_dataset)*0.2)))

from torch.utils.data import DataLoader
batch_size = 100
train_dl = DataLoader(siamse_dataset_train, batch_size = batch_size, shuffle = True)
test_dl = DataLoader(siamse_dataset_test, batch_size = batch_size, shuffle = True)

1327
331


In [7]:
embedding_net = EmbeddingNet()
model = TripletNet(embedding_net)
criterion = nn.TripletMarginLoss(margin=1, p=2)
optimizer = torch.optim.Adagrad(model.parameters())
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
# wandb.login(host='wandb.ai' relogin=True)

In [8]:
optimizer.state_dict

<bound method Optimizer.state_dict of Adagrad (
Parameter Group 0
    eps: 1e-10
    initial_accumulator_value: 0
    lr: 0.01
    lr_decay: 0
    weight_decay: 0
)>

In [10]:
import wandb
wandb.login()

wandb: Currently logged in as: metpinc (use `wandb login --relogin` to force relogin)


True

In [11]:
wandb.init(project="triplet_siamse_network", config={
    "learning_rate": 0.01,
    "architecture": "SNN"
})
config = wandb.config

In [ ]:
for epoch in range(10):
    print('\n\nEpoch', epoch)

    print('train')
    
    train_loss_history = []
    train_accuracy_history = []

    test_accuracy_history = []
    test_loss_history = []
    
    for i, (anchor, positive, negative) in (enumerate(train_dl)):
        optimizer.zero_grad()
        
        pred = model.forward(anchor, positive, negative)
        pred_anchor, pred_positive, pred_negative = pred
        loss = criterion(pred_anchor, pred_positive, pred_negative)

        loss.backward() #count gradients
        optimizer.step() #update weights
        scheduler.step(loss)

        cur_accuracy = accuracy(pred_anchor, pred_positive, pred_negative)
        print(f'  [{i}/{len(train_dl)}] acc {cur_accuracy} loss {loss}')
        
        train_accuracy_history.append(cur_accuracy)
        train_loss_history.append(loss)
        wandb.log({"train/accuracy": cur_accuracy, "train/loss": loss})

    print('train accuracy: ', mean(train_accuracy_history))
    print('train loss: ', sum(train_loss_history)/len(train_loss_history))


    print('test')
    with torch.no_grad():
        for i, (anchor, positive, negative) in enumerate(test_dl):
            pred = model.forward(anchor, positive, negative)
            pred_anchor, pred_positive, pred_negative = pred
            loss = criterion(pred_anchor, pred_positive, pred_negative)
            cur_accuracy = accuracy(pred_anchor, pred_positive, pred_negative)
            test_accuracy_history.append(cur_accuracy)
            wandb.log({"test/accuracy":cur_accuracy})
            wandb.log({"test/loss": loss})
            print(f'  [{i}/{len(train_dl)}] acc {cur_accuracy} loss {loss}')
            
    print('test accuracy: ', mean(test_accuracy_history))




Epoch 0
train


c:\users\май\appdata\local\programs\python\python36\lib\site-packages\torch\autograd\__init__.py:132: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  allow_unreachable=True)  # allow_unreachable flag


  [0/14] acc 0.76 loss 0.7518860697746277
  [1/14] acc 0.75 loss 1.3860619068145752
  [2/14] acc 0.65 loss 1.3839036226272583
  [3/14] acc 0.61 loss 1.6354833841323853
  [4/14] acc 0.69 loss 0.9536917805671692
  [5/14] acc 0.7 loss 0.9747981429100037
  [6/14] acc 0.75 loss 0.6950533986091614
  [7/14] acc 0.77 loss 0.55531907081604
  [8/14] acc 0.84 loss 0.486968070268631
  [9/14] acc 0.77 loss 0.6205918788909912
  [10/14] acc 0.76 loss 0.6764528751373291
  [11/14] acc 0.82 loss 0.4739980399608612
  [12/14] acc 0.78 loss 0.6145699620246887
  [13/14] acc 0.7407407407407407 loss 0.5311856865882874
train accuracy:  0.7421957671957672
train loss:  tensor(0.8386, grad_fn=<DivBackward0>)
test
  [0/14] acc 0.66 loss 1.4268813133239746
  [1/14] acc 0.72 loss 0.9612684845924377
  [2/14] acc 0.71 loss 1.1484047174453735
  [3/14] acc 0.78125 loss 0.7590048313140869
test accuracy:  0.7178125


Epoch 1
train
  [0/14] acc 0.77 loss 0.8372530341148376
  [1/14] acc 0.77 loss 0.5707618594169617
  [2/14]

In [8]:
import torchvision.transforms.functional as F
pic0 = F.to_pil_image(siamse_dataset[0][0])
pic1 = F.to_pil_image(siamse_dataset[0][1])
pic2 = F.to_pil_image(siamse_dataset[0][2])
pics_pool = [pic1, pic2]

In [9]:
model.get_similars(pic0, pics_pool)

2it [00:01,  1.64it/s]
2it [00:01,  1.17it/s]


[0, 1]